<font color='green' size= 6 ><b>Extract and Transform the data</b></font>

In [ ]:
# create lists
season_list = []
episod_season_list = []
episod_list= []
title_list = []
rating_list = []
day_list= []
month_list= []
year_list = []
link_list = []

link =  ("https://www.imdb.com/title/tt1528406/episodes/?season=" + str(season_number))

write_to_file(log_file, f"Extracting {number_ep_season_to_extract} episod(s) from season {season_number} in the following {link}")

soup = create_soup(link)
articles = soup.find_all('article')
episod = number_ep_database

for i in range((nb_ep_season[season_number-1] - number_ep_season_to_extract) , nb_ep_season[season_number-1]):
    
    try:
        write_to_file(log_file, f"Extracting episod {i+1}")
        
        # Season, Episod of season, Episod, Title and Link
        article = articles[i].find("div",class_="ipc-title ipc-title--base ipc-title--title ipc-title-link-no-icon ipc-title--on-textPrimary sc-ccd6e31b-8 fPPYex")
        season = int(article.text.split(" ∙ ")[0].split(".E")[0][1:])
        season_list.append(season)

        if season_number in [1,5,6,7,8]:
            episod_season = int(article.text.split(" ∙ ")[0].split(".E")[1])
        else:
            episod_season = i + 1
        episod_season_list.append(episod_season)

        episod = episod + 1
        episod_list.append(episod)

        title = article.text.split(" ∙ ")[1].replace(",","")
        title_list.append(title)

        link = extract_link(article.a.get("href"))
        link_list.append(link)

        #Rating
        rate = articles[i].find("span",class_="ipc-rating-star--rating").text.replace(",",".")
        rating_list.append(float(rate))
     
        # Date
        date = articles[i].find("span",class_="sc-ccd6e31b-10 dYquTu").text
        day,month,year = extract_date(date)
        day_list.append(day)
        month_list.append(month)
        year_list.append(year)
     
    except IndexError:
        pass         

    except WebDriverException as E: 
        write_to_file(log_file, f"Extracting episod {i+1} failed")

    i = i+1

<font color='green' size= 6 ><b>Load the data</b></font>

In [ ]:
# Create DataFrame of the data
data = {"Episod":episod_list,"Season":season_list,"Episod_season":episod_season_list, "Title":title_list, "Day":day_list, "Month":month_list, "Year":year_list, "Rate":rating_list, "link":link_list}
df = pd.DataFrame(data)

# Check if all the data have been successfully extracted
if df.shape[0] == number_ep_season_to_extract:
    write_to_file(log_file,f"Extracted {number_ep_season_to_extract} episod successfuly")
else:
    write_to_file(log_file,f"Extraction failed")

In [ ]:
# Load the data into a csv file
df.to_csv(csv_file,index=False,mode= "a")

In [ ]:
# Load the data into MySQL database

write_to_file(log_file,f"Sending {len(df)} rows to MySQL table '{table_name}'.")
try:
    # Create a list of column names from the DataFrame
    columns = ', '.join(df.columns)

    # Create a placeholders string for the SQL query
    placeholders = ', '.join(['%s'] * len(df.columns))

    # Create the SQL query
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Execute the SQL query with the DataFrame values
    cursor.executemany(sql, df.values.tolist())

    # Commit the changes to the database
    cnx.commit()

    write_to_file(log_file,f"Successfully sent {len(df)} rows to MySQL table '{table_name}'.")
except Exception as e:
    write_to_file(log_file, f"There is an error : {e}")
    # Close the database cursor and connection
    if cursor:
        cursor.close()
    if cnx:
        cnx.close()
    raise